In [1]:
import pandas as pd
from os import path, environ
from dotenv import load_dotenv
from unidecode import unidecode

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [3]:
input_dir = path.join('dados', 'urbanismo')

In [ ]:
filename = path.join(input_dir, 'orcamento_habitacao_original.csv')
df_orcamento = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_orcamento

In [ ]:
filename = path.join(input_dir, 'orcamento_regionalizado_habitacao_original.csv')
df_orcamento_r = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_orcamento_r

In [ ]:
filename = path.join(input_dir, 'pdm_meta_12_original.csv')
df_meta_12 = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_meta_12

In [ ]:
filename = path.join(input_dir, 'his_entregue_original.csv')
df_his = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_his

In [ ]:
filename = path.join(input_dir, 'tpu_emitido_original.csv')
df_tpu = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_tpu

# Transformação e padronização

Nos indicadores de habitação, apenas o ano e as subprefeituras são presentes em vários arquivos. Mas, além disso, os arquivos precisarão de tratamentos específicos. Vamos começar com os mais simples, onde é necessário apenas padronizar as subprefeituras.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs = pd.read_csv(url_subs)
df_subs

In [ ]:
df_subs = df_subs[['sub.CODIGO', 'sub.NOME']]
df_subs

## PdM - Meta 12: Prover 49.000 moradias de interesse social

In [ ]:
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.loc[:, ['Subprefeitura', '2021', '2022', '2023', '2024']]

df_meta_12

In [ ]:
df_meta_12 = df_meta_12.melt('Subprefeitura',
                var_name='ano',
                value_name='qtd_unidades_acumulado')

df_meta_12

Como os valores foram divulgados no acumulado entre 2021 e 2024, precisamos ajustar para o incremento de cada ano antes de carregar os dados no Qlik.

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12.groupby('Subprefeitura')['qtd_unidades_acumulado'].diff()
df_meta_12

In [ ]:
df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades'] = (
    df_meta_12.loc[df_meta_12['ano']=='2021', 'qtd_unidades_acumulado'])
df_meta_12

In [ ]:
df_meta_12 = df_meta_12.drop(columns='qtd_unidades_acumulado')
df_meta_12

Finalmente, vamos criar uma coluna com os nomes padronizados de subprefeituras.

In [ ]:
subs_meta_12 = df_meta_12['Subprefeitura'].apply(unidecode).unique().tolist()
subs_meta_12.sort()
subs_meta_12

In [ ]:
subs_qlik = df_subs['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

In [ ]:
len(subs_meta_12)

Vemos que existem 3 subprefeituras faltantes no dataframe da meta 12. Vamos avaliar quais podem ser. Numa inspeção detalhada vemos que faltam `ARICANDUVA-FORMOSA-CARRAO`, `SAO MIGUEL` e `VILA MARIANA`. Vamos criar uma cópia da lista de subs do qlik adaptada à meta 12.

In [ ]:
subs_qlik_meta_12 = subs_qlik.copy()
subs_qlik_meta_12.remove('ARICANDUVA-FORMOSA-CARRAO')
subs_qlik_meta_12.remove('SAO MIGUEL')
subs_qlik_meta_12.remove('VILA MARIANA')
subs_qlik_meta_12

In [ ]:
mapper_meta_12 = {
    o: q
    for o, q in zip(subs_meta_12, subs_qlik_meta_12)
}

mapper_meta_12

In [ ]:
df_meta_12.insert(1,
                  'sub.NOME',
                  df_meta_12['Subprefeitura'].apply(unidecode).map(mapper_meta_12))
df_meta_12

In [ ]:
df_meta_12['qtd_unidades'] = df_meta_12['qtd_unidades'].astype(int)
df_meta_12

## Produção de habitação de interesse social

In [ ]:
df_his

In [ ]:
subs_his = df_his['região'].apply(unidecode).unique().tolist()
subs_his.sort()
subs_his

In [ ]:
mapper_his = {
    s: q for s, q in zip(subs_his, subs_qlik)
}

mapper_his

In [ ]:
df_his.insert(1,
                  'sub.NOME',
                  df_his['região'].apply(unidecode).map(mapper_his))
df_his

In [ ]:
df_his['qtd_unidades'] = df_his['qtd_unidades'].astype(int)
df_his